In [1]:
import numpy  as np
import h5py
import os

In [3]:
# Will process a txt/.ply/.pcd  file containing xyz coords of points and returns a list of
# numpy arrays of each point. Assumes that the input file is the format that
# CloudCompare exports point files as
def process_pt_file(path):
    data = []
    # Process single point cloud file
    pt_file = open(path, 'r')
    # Pass over header lines
    line='placeholder'
    num_pts=0
    if(path.endswith( '.txt')):  
        header = pt_file.readline()
        num_pts = int(pt_file.readline())
        line = pt_file.readline()
    elif (path.endswith( '.ply')):  
        while line:
            line_list = line.split()
            line = pt_file.readline()
            if('element' in line_list[0] and 'vertex' in line_list[1] ):
                num_pts = int(line_list[2])
            if('end_header' in line_list[0]):
                break
    elif (path.endswith( '.pcd')):
        while line:
            line_list = line.split()
            line = pt_file.readline()
            if('POINTS' in line_list[0]   ):
                num_pts = int(line_list[1])
            if('DATA' in line_list[0] and 'ascii' in line_list[1]):
                break  
    previousEleNum = 0
    currentEleNum = 0
   
    while line:
        line_list = line.split()
        data.append(np.array([float(line_list[0]), float(line_list[1]), float(line_list[2])]))
        line = pt_file.readline()
        previousEleNum=currentEleNum
        currentEleNum=len(line.split())  
        if(currentEleNum>6 or (previousEleNum>0 and currentEleNum != previousEleNum)):
            break
       
       
    # Check number of points
    data = np.array(data)
    checkStr="Number of processed points %d   header number of points %d"%(np.shape(data)[0],num_pts)
    print(checkStr)
    assert num_pts == np.shape(data)[0], checkStr
    return data



In [4]:
import numpy as np
from sklearn.preprocessing import normalize
path='2.ply'
x1=process_pt_file(path)
z1 = normalize(x1)
z1 = z1.astype('float32')
z1

Number of processed points 12054   header number of points 12054


array([[ 0.        ,  0.51843786, -0.8551153 ],
       [ 0.        ,  0.5329102 , -0.8461718 ],
       [-0.00709714,  0.5205727 , -0.85378784],
       ...,
       [ 0.2628656 ,  0.1624599 ,  0.9510565 ],
       [ 0.5257311 ,  0.        ,  0.8506508 ],
       [ 0.2628656 , -0.1624599 ,  0.9510565 ]], dtype=float32)

In [5]:
import numpy as np
from sklearn.preprocessing import normalize
path='5000_polygon_sphere_100mm.ply'
x2=process_pt_file(path)
z2 = normalize(x2)
z2 = z2.astype('float32')
z2

Number of processed points 4902   header number of points 4902


array([[-0.46399075, -0.22485436,  0.8568273 ],
       [-0.4697078 , -0.17316128,  0.865673  ],
       [-0.47235802, -0.17298687,  0.86426467],
       ...,
       [-0.46432745, -0.1741337 , -0.8683764 ],
       [-0.4670275 , -0.17354432, -0.8670454 ],
       [-0.46399075, -0.22485436, -0.8568273 ]], dtype=float32)

In [6]:
import numpy as np
from sklearn.preprocessing import normalize
path='Ball with a surprise_new.ply'
x3=process_pt_file(path)
z3 = normalize(x3)
z3 = z3.astype('float32')
z3

Number of processed points 7588   header number of points 7588


array([[0.80331385, 0.4342515 , 0.407569  ],
       [0.8026518 , 0.433359  , 0.409817  ],
       [0.8164966 , 0.4082483 , 0.4082483 ],
       ...,
       [0.7625665 , 0.4891107 , 0.42339468],
       [0.75763047, 0.49269047, 0.42807963],
       [0.75880444, 0.48982716, 0.4292845 ]], dtype=float32)

In [7]:
import numpy as np
from sklearn.preprocessing import normalize
path='sphere12.ply'
x4=process_pt_file(path)
z4 = normalize(x4)
z4 = z4.astype('float32')
z4

Number of processed points 43512   header number of points 43512


array([[-0.05085554,  0.16123794,  0.9856044 ],
       [-0.0358876 ,  0.16094686,  0.98631036],
       [-0.0241993 ,  0.1634319 ,  0.9862578 ],
       ...,
       [ 0.03618108, -0.16127436, -0.98624617],
       [ 0.05089812, -0.16124529, -0.985601  ],
       [ 0.00274559, -0.00274606, -0.9999925 ]], dtype=float32)

In [8]:
import numpy as np
from sklearn.preprocessing import normalize
path='sphrcoewr.ply'
x5=process_pt_file(path)
z5 = normalize(x5)
z5 = z5.astype('float32')
z5

Number of processed points 3649   header number of points 3649


array([[0.06241392, 0.76336676, 0.64294297],
       [0.06570954, 0.75040334, 0.65770596],
       [0.21338479, 0.7099585 , 0.67113775],
       ...,
       [0.26524907, 0.67077327, 0.69260824],
       [0.21764313, 0.6651667 , 0.7142722 ],
       [0.27561328, 0.66730374, 0.6919126 ]], dtype=float32)

In [9]:
total_data=np.concatenate((z1, z2,z3,z4,z5), axis=0)

In [10]:
len(total_data)

71705

In [11]:
def make_pt_clouds(total_data, num_clouds, num_pts, filename,randomizePoint=True):
    f = h5py.File(filename, 'w')
   
    if(randomizePoint):
        rand_inds = np.random.randint(np.shape(total_data)[0], size = num_pts)
    else:
        rand_inds=range(num_pts)
    pt_data = np.reshape(total_data[rand_inds, :], (1, num_pts, 3))
    pt_label = np.array([[40]]*num_clouds)
    for i in range(num_clouds - 1):
        rand_inds = np.random.randint(np.shape(total_data)[0], size = num_pts)
        curr_cloud = np.reshape(total_data[rand_inds, :], (1, num_pts, 3))
        pt_data = np.concatenate((pt_data, curr_cloud), axis=0)
    f.create_dataset('data', data=pt_data)
    f.create_dataset('label', data=pt_label)
    f.close()
    print("pt_data shape: ", np.shape(pt_data))


In [12]:
make_pt_clouds(total_data, 1648, 2048, 'ply_data_test2.h5', randomizePoint=True)

pt_data shape:  (1648, 2048, 3)


In [14]:
import h5py
filename = 'ply_data_test2.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]
b_group_key = list(f.keys())[1]
# Get the data
data = list(f[a_group_key])
label = list(f[b_group_key])

Keys: KeysView(<HDF5 file "ply_data_test2.h5" (mode r)>)


In [15]:
list(f.keys())

['data', 'label']

In [16]:
dset = f['data']

In [17]:
dset.shape

(1648, 2048, 3)

In [18]:
data

[array([[-0.47224304,  0.8696867 , -0.14363681],
        [ 0.87640625,  0.41605493, -0.24250841],
        [ 0.63391906, -0.77338237, -0.00512599],
        ...,
        [-0.86418873,  0.2892536 , -0.41171613],
        [ 0.7177214 ,  0.11425046,  0.68689364],
        [-0.45881572, -0.86723375,  0.1933747 ]], dtype=float32),
 array([[-0.60634327, -0.79409903, -0.04188806],
        [ 0.04241661,  0.3487218 , -0.93626595],
        [ 0.8197852 ,  0.5725804 ,  0.01019266],
        ...,
        [-0.3833869 , -0.0528149 ,  0.92207646],
        [ 0.89144564, -0.0725826 ,  0.4472767 ],
        [-0.07104623,  0.20775679,  0.975597  ]], dtype=float32),
 array([[ 0.1440727 ,  0.45928186,  0.8765291 ],
        [-0.3741578 ,  0.92600465,  0.05021301],
        [ 0.38531882,  0.6990791 ,  0.6023436 ],
        ...,
        [ 0.28678095,  0.18287122,  0.94038016],
        [ 0.16535851, -0.70329064, -0.6914035 ],
        [-0.66865075, -0.74062157, -0.06622628]], dtype=float32),
 array([[ 0.7636047 ,  0.470